# Aprendizaje Profundo - Práctico (Parte II)

Equipo: Mariana Pereyra - Adrián Zelaya

____

Algunas consideraciones a tener en cuenta para estructurar el trabajo:

1. Hacer un preprocesamiento de los datos (¿Cómo vamos a representar los datos de entrada y las categorías?).
2. Tener un manejador del dataset (alguna clase o función que nos divida los datos en batches).
3. Crear una clase para el modelo que se pueda instanciar con diferentes hiperparámetros
4. Hacer logs de entrenamiento (reportar tiempo transcurrido, iteraciones/s, loss, accuracy, etc.). Usar MLFlow.
5. Hacer un gráfico de la función de loss a lo largo de las epochs. MLFlow también puede generar la gráfica.
6. Reportar performance en el conjunto de test con el mejor modelo entrenado. La métrica para reportar será balanced accuracy.

## Objetivo

En este práctico implementaremos una red convolucional (CNN) que asigne una categoría dado un título.



### Descarga de datos 




Descargamos los datos localmente usando los siguientes comandos:

```
!curl -L https://cs.famaf.unc.edu.ar/\~ccardellino/resources/diplodatos/meli-challenge-2019.tar.bz2 -o ./data/meli-challenge-2019.tar.bz2
!tar jxvf ./data/meli-challenge-2019.tar.bz2 -C ./data/
```

Luego los subimos a una carpeta de drive para utilizarlos desde Google Colab.


In [ ]:
!curl -L https://cs.famaf.unc.edu.ar/\~ccardellino/resources/diplodatos/glove.6B.50d.txt.gz -o glove.6B.50d.txt.gz

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100 65.9M  100 65.9M    0     0  6854k      0  0:00:09  0:00:09 --:--:-- 11.7M


In [ ]:
from google.colab import drive 
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
path_meli_train = "/content/drive/MyDrive/Diplomatura/Aprendizaje Profundo/meli-challenge-2019/spanish.train.jsonl.gz"
path_meli_test = "/content/drive/MyDrive/Diplomatura/Aprendizaje Profundo/meli-challenge-2019/spanish.test.jsonl.gz"
path_glove = "/content/glove.6B.50d.txt.gz"

### Librerías

Instalamos e importamos las librerías necesarias para nuestro análisis

In [ ]:
!pip install gensim --upgrade 
!pip install mlflow 

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 24.1 MB 1.5 MB/s 
  Attempting uninstall: gensim
    Found existing installation: gensim 3.6.0
    Uninstalling gensim-3.6.0:
      Successfully uninstalled gensim-3.6.0
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 17.0 MB 58 kB/s 
     |████████████████████████████████| 147 kB 67.4 MB/s 
     |████████████████████████████████| 79 kB 9.4 MB/s 
     |████████████████████████████████| 77 kB 6.0 MB/s 
     |████████████████████████████████| 182 kB 65.3 MB/s 
     |████████████████████████████████| 209 kB 64.5 MB/s 
     |████████████████████████████████| 78 kB 7.2 MB/s 
     |████████████████████████████████| 140 kB 72.6 MB/s 
     |████████████████████████████████| 62 kB 1.8 MB/s 
     |████████████████████████████████| 55 kB 4.0 MB/s 
     |█████████████████

In [ ]:
import csv
import functools
import numpy as np
import json
import gzip
import mlflow
import pandas as pd
import tempfile
import torch
import torch.optim as optim
import torch.nn as nn
import torch.nn.functional as F

from gensim import corpora
from gensim.parsing import preprocessing
from sklearn.model_selection import train_test_split
from sklearn.metrics import f1_score, balanced_accuracy_score
from torch.utils.data import Dataset, DataLoader, IterableDataset
from tqdm.notebook import tqdm, trange

from gensim.models import KeyedVectors

In [ ]:
# Check if we have a GPU available
use_cuda = torch.cuda.is_available()
print(use_cuda)
device = torch.device('cuda') if use_cuda else torch.device('cpu')

True


## Dataset

Nos basaremos en el dataset del MeLi Data Challenge 2019 para trabajar en el problema de clasificación de texto de este práctico.

El dataset contiene información acerca de los títulos de publicaciones, categorías de las mismas, información de idioma y confiabilidad de la anotación. Cuenta con anotaciones de títulos para 632 categorías distintas.

El dataset también cuenta con una partición de test que está compuesta de 636,680 de ejemplos con las mismas categorías (aunque no necesariamente la misma distribución).

También hay datos en idioma portugués, aunque para este práctico nos centraremos solamente en el idioma español.

Para trabajar con el conjuntos de datos en PyTorch se podría utilizar la clase  IterableDataset. Esta elección se debe a que esta clase permite trabajar más eficientemente con un conjunto de datos grade, como es el caso del MeLi Challenge. Pero por cuestiones de simplicidad usaremos Dataset.

In [ ]:
class MeLiChallengeDataset(Dataset):
    def __init__(self, dataset, transform=None):
        self.dataset = dataset
        self.transform = transform
    
    def __len__(self):
        return self.dataset.shape[0]

    def __getitem__(self, item):
        if torch.is_tensor(item):
            item = item.to_list()
        
        item = {
            "data": self.dataset.loc[item, "title"],
            "target": self.dataset.loc[item, "category"]
        }
        
        if self.transform:
            item = self.transform(item)
        
        return item

Inspeccionamos la data de train usando Pandas

In [ ]:
train_df = pd.concat([x for x in pd.read_json(path_meli_train, lines=True, chunksize=100000)], ignore_index=True)
train_df.head()

,language,label_quality,title,category,split,tokenized_title,data,target,n_labels,size
0,spanish,reliable,Casita Muñecas Barbies Pintadas,DOLLHOUSES,train,"[casita, muñecas, barbies, pintadas]","[50001, 2, 50000, 3]",0,632,4895280
1,spanish,unreliable,Neceser Cromado Holográfico,TOILETRY_BAGS,train,"[neceser, cromado, holográfico]","[6, 4, 5]",1,632,4895280
2,spanish,unreliable,Funda Asiento A Medida D20 Chevrolet,CAR_SEAT_COVERS,train,"[funda, asiento, medida, chevrolet]","[9, 7, 10, 8]",2,632,4895280
3,spanish,unreliable,Embrague Ford Focus One 1.8 8v Td (90cv) Desde...,AUTOMOTIVE_CLUTCH_KITS,train,"[embrague, ford, focus, one]","[11, 13, 12, 14]",3,632,4895280
4,spanish,unreliable,Bateria Panasonic Dmwbcf10 Lumix Dmc-fx60n Dmc...,CAMERA_BATTERIES,train,"[bateria, panasonic, dmwbcf, lumix, dmc, fxn, ...","[15, 19, 17, 18, 16, 1, 1, 1]",4,632,4895280


In [ ]:
X_train, y_train = train_df[['title']], train_df[['category']]

display(X_train.head())

display(y_train.head())

,title
0,Casita Muñecas Barbies Pintadas
1,Neceser Cromado Holográfico
2,Funda Asiento A Medida D20 Chevrolet
3,Embrague Ford Focus One 1.8 8v Td (90cv) Desde...
4,Bateria Panasonic Dmwbcf10 Lumix Dmc-fx60n Dmc...


,category
0,DOLLHOUSES
1,TOILETRY_BAGS
2,CAR_SEAT_COVERS
3,AUTOMOTIVE_CLUTCH_KITS
4,CAMERA_BATTERIES


Inspeccionamos la data de Test

In [ ]:
test_df = pd.concat([x for x in pd.read_json(path_meli_test, lines=True, chunksize=100000)], ignore_index=True)
test_df.head()

,language,label_quality,title,category,split,tokenized_title,data,target,n_labels,size
0,spanish,reliable,Mochilas Maternales Bolsos Bebe Simil Cuero Ma...,DIAPER_BAGS,test,"[mochilas, maternales, bolsos, bebe, simil, cu...","[5650, 5271, 5268, 915, 2724, 375, 37363]",318,632,63680
1,spanish,reliable,Bolso Maternal/bebe Incluye Cambiador + Correa...,DIAPER_BAGS,test,"[bolso, maternal, bebe, incluye, cambiador, co...","[502, 2742, 915, 3031, 2740, 1840, 4635]",318,632,63680
2,spanish,reliable,Mochila Maternal Land + Gancho Envio Gratis-cc,DIAPER_BAGS,test,"[mochila, maternal, land, gancho, envio, gratis]","[337, 2742, 2741, 3303, 211, 1429]",318,632,63680
3,spanish,reliable,Bolso Maternal Moderno Con Cambiador Y Correa ...,DIAPER_BAGS,test,"[bolso, maternal, moderno, cambiador, correa, ...","[502, 2742, 2983, 2740, 1840, 476, 2990]",318,632,63680
4,spanish,reliable,Bolso Maternal Moderno Con Cambiador Y Correa ...,DIAPER_BAGS,test,"[bolso, maternal, moderno, cambiador, correa, ...","[502, 2742, 2983, 2740, 1840, 4635]",318,632,63680


In [ ]:
test_df_reduced = test_df.iloc[:10000,:]

In [ ]:
X_test, y_test = test_df[['title']], test_df[['category']]

display(X_test.head())
display(y_test.head())

,title
0,Mochilas Maternales Bolsos Bebe Simil Cuero Ma...
1,Bolso Maternal/bebe Incluye Cambiador + Correa...
2,Mochila Maternal Land + Gancho Envio Gratis-cc
3,Bolso Maternal Moderno Con Cambiador Y Correa ...
4,Bolso Maternal Moderno Con Cambiador Y Correa ...


,category
0,DIAPER_BAGS
1,DIAPER_BAGS
2,DIAPER_BAGS
3,DIAPER_BAGS
4,DIAPER_BAGS


Chequeamos que las categorías en train y test sean las mismas:

In [ ]:
n_categories = len(y_train.category.unique())

In [ ]:
len(y_train.category.unique())

632

In [ ]:
len(y_test.category.unique())

632

In [ ]:
len(set(y_train.category.unique()).intersection(set(y_test.category.unique())))

632

## Preprocesamiento

Vamos a utilizar el siguiente procesador, visto en clase (notebook 4). Este se encargará de preprocesar el texto (i.e. normalizarlo) y transformará las palabras en índices de un diccionario para luego poder pasar una secuencia de palabras para buscar en la matriz de embeddings y así permitir mayor manipulación de los embeddings (en lugar de utilizar embeddings fijos).

Vamos a estar trabajando con la librería gensim previamente importada para el procesamiento del lenguaje natural.

En cuanto los pasos de preprocesamiento realizados, permiten uniformizar el texto de la mejor manera posible y contemplan lo siguiente:

* Pasar el texto a minúsculas (lower case).
* Remover tags de HTML que puedan estar embebidas en el texto.
* Remover símbolos de puntuación.
* Limpiar espacios: remover espacios múltiples y espacios en exceso al inicio o al final del texto ('title').
* Remover valores numéricos.
* Remover stopwords.

In [ ]:
class RawDataProcessor:
    def __init__(self, 
                 dataset, 
                 ignore_header=True, 
                 filters=None, 
                 vocab_size=50000):
        if filters:
            self.filters = filters
        else:
            self.filters = [ #We set some filters
                lambda s: s.lower(),
                preprocessing.strip_tags,
                preprocessing.strip_punctuation,
                preprocessing.strip_multiple_whitespaces,
                preprocessing.strip_numeric,
                preprocessing.remove_stopwords,
                preprocessing.strip_short,
            ]
        # Create dictionary based on all the reviews (with corresponding preprocessing and filters)
        # The dictionary has idx as a key and word as a value
        # For example one element could be (0, 'accustomed') or (1, 'agenda')
        self.dictionary = corpora.Dictionary(
            dataset["title"].map(self._preprocess_string).tolist()
        )
        
        # Filter the dictionary and compactify it (make the indices continous)
        self.dictionary.filter_extremes(no_below=2, no_above=1, keep_n=vocab_size)
        self.dictionary.compactify()
        # Add a couple of special tokens
        self.dictionary.patch_with_special_tokens({
            "[PAD]": 0, #The padding token
            "[UNK]": 1  # The unknown token
        })
        
        self.idx_to_target = sorted(dataset["category"].unique())
        self.target_to_idx = {t: i for i, t in enumerate(self.idx_to_target)}
        

    def _preprocess_string(self, string):
        return preprocessing.preprocess_string(string, filters=self.filters)

    def _sentence_to_indices(self, sentence):
        return self.dictionary.doc2idx(sentence, unknown_word_index=1)
    
    def encode_data(self, data):
        return self._sentence_to_indices(self._preprocess_string(data))
    
    def encode_target(self, target):
        return self.target_to_idx[target]
    
    def __call__(self, item):
        #Encodeamos tanto los datos como los targets, diferenciandos los casos cuando son strings y cuando no
        if isinstance(item["data"], str): 
            data = self.encode_data(item["data"])
        else:
            data = [self.encode_data(d) for d in item["data"]]
        
        if isinstance(item["target"], str):
            target = self.encode_target(item["target"])
        else:
            target = [self.encode_target(t) for t in item["target"]]
        
        return {
            "data": data,
            "target": target
        }

## Lectura de datos

Leemos los datos del MeLiChallenge para utilizarlos en el entrenamiento y evaluación del modelo. En este caso no es necesario dividirlos entre test y training, ya que los mismos ya se encuentran divididos como tal desde los archivos de entrada.

In [ ]:
preprocess_train = RawDataProcessor(train_df)
train_dataset = MeLiChallengeDataset(train_df, transform=preprocess_train)

preprocess_test = RawDataProcessor(test_df)
test_dataset = MeLiChallengeDataset(test_df, transform=preprocess_test)

print(f"Datasets loaded with {len(train_dataset)} training elements and {len(test_dataset)} test elements")
print(f"Sample train element:\n{train_dataset[0]}")

Datasets loaded with 4895280 training elements and 63680 test elements
Sample train element:
{'data': [50001, 2, 50000, 3], 'target': 188}


## Collation function

Como en este caso trabajamos con secuencias de palabras (representadas por sus índices en un vocabulario), cuando queremos buscar un batch de datos, el DataLoader de PyTorch espera que los datos del batch tengan la misma dimensión (para poder llevarlos todos a un tensor de dimensión fija). Esto lo podemos lograr mediante el parámetro de collate_fn. En particular, esta función se encarga de tomar varios elementos de un Dataset y combinarlos de manera que puedan ser devueltos como un tensor de PyTorch.

En nuestro caso, como se vio en clase, definimos un módulo PadSequences que toma un valor mínimo, opcionalmente un valor máximo y un valor de relleno (pad) y dada una lista de secuencias, devuelve un tensor con padding sobre dichas secuencias.

In [ ]:
class PadSequences:
    def __init__(self, pad_value=0, max_length=None, min_length=1):
        assert max_length is None or min_length <= max_length
        self.pad_value = pad_value
        self.max_length = max_length
        self.min_length = min_length

    def __call__(self, items):
        data, target = list(zip(*[(item["data"], item["target"]) for item in items]))
        seq_lengths = [len(d) for d in data]

        if self.max_length:
            max_length = self.max_length
            seq_lengths = [min(self.max_length, l) for l in seq_lengths]
        else:
            max_length = max(self.min_length, max(seq_lengths))

        data = [d[:l] + [self.pad_value] * (max_length - l)
                for d, l in zip(data, seq_lengths)]
            
        return {
            "data": torch.LongTensor(data),
            "target": torch.LongTensor(target)
        }

## DataLoaders

Ya habiendo definido nuestros conjuntos de datos y nuestra collation_fn, podemos definir nuestros DataLoader, uno para entrenamiento y otro para evaluación.

Notar que shuffle = False para el caso del loader de evaluación, dado que no queremos mezclar los valores de evaluación cada vez que evaluamos porque al evaluar mediante mini-batchs nos puede generar inconsistencias.

A diferencia del Práctico I, en este caso aquí también definimos el número y el tamaño de los filtros que se van a utilizar en la red convolucional.

In [ ]:
EPOCHS = 3
FILTERS_COUNT = 100
FILTERS_LENGTH = [2, 3, 4]

pad_sequences = PadSequences(min_length=max(FILTERS_LENGTH))
train_loader = DataLoader(train_dataset, 
                          batch_size=128, 
                          shuffle=True,
                          collate_fn=pad_sequences, 
                          drop_last=False)
test_loader = DataLoader(test_dataset, 
                         batch_size=128, 
                         shuffle=False,
                         collate_fn=pad_sequences, 
                         drop_last=False)

## El modelo de clasificación

Para clasificación utilizaremos una red convolucional (CNN) basados en lo que trabajamos en clases. La misma consistía de: tres convoluciones con filtros de 2 x 100, 3 x 100 y 4 x 100; una capa fully connected y una capa de salida. Además entre las capas convolucionales definimos capas de pooling (global max pooling).


En particular, tenemos la capa de Embeddings que es rellenada con los valores de embeddings preentrenados (los de Glove en este caso).

En el caso de la capa de salida, se asume que el número de nodos se corresponde con el número de categorías posibles, en este caso 632. Los valores de salida son continuos (y van de 0 a 1). Luego, para determinar la clase predicha por el modelo, por fuera de la arquitectura de la red neuronal tomamos el máximo de todos los valores de salida y asignamos al nodo con el valor máximo un 1, y al resto 0 (ver sección de MLflow).

In [ ]:
class MeLiChallengeClassifier(nn.Module):
    def __init__(self, 
                 pretrained_embeddings_path, 
                 dictionary,
                 vector_size,
                 freeze_embedings,
                 output_size):
        super().__init__()
        
        # Inicializamos la matriz de embeddings
        embeddings_matrix = torch.randn(len(dictionary), vector_size)
        embeddings_matrix[0] = torch.zeros(vector_size)
        
        #Trabajamos con los embeddings preentrenados
        with gzip.open(pretrained_embeddings_path, "rt") as fh:
            for line in fh:
                word, vector = line.strip().split(None, 1)
                if word in dictionary.token2id:
                    embeddings_matrix[dictionary.token2id[word]] =\
                        torch.FloatTensor([float(n) for n in vector.split()])
        
        # Los guardamos en la variable embeddings
        self.embeddings = nn.Embedding.from_pretrained(embeddings_matrix,
                                                       freeze=freeze_embedings,
                                                       padding_idx=0)
        self.convs = []
        for filter_lenght in FILTERS_LENGTH:
            self.convs.append(
                nn.Conv1d(vector_size, FILTERS_COUNT, filter_lenght) #(in_channels, out_channels, kernel_size)
            )
        self.convs = nn.ModuleList(self.convs)
        self.fc = nn.Linear(FILTERS_COUNT * len(FILTERS_LENGTH), 128)
        self.output = nn.Linear(128, output_size)
        self.vector_size = vector_size
    
    @staticmethod
    def conv_global_max_pool(x, conv):
        return F.relu(conv(x).transpose(1, 2).max(1)[0])
    
    def forward(self, x):
        x = self.embeddings(x).transpose(1, 2)  
        x = [self.conv_global_max_pool(x, conv) for conv in self.convs]
        x = torch.cat(x, dim=1)
        x = F.relu(self.fc(x))
        x = torch.sigmoid(self.output(x))
        return x

## Experimento de MLflow

Utilizamos MLflow para hacer un mejor seguimiento del experimento y registramos la siguiente información: parámetros, métricas, error y artefactos (resultados de la predicción).

Seteamos los parámetros de antemano e hicimos las siguientes modificaciones respecto a la versión vista en clases:

* Utilizamos balanced accuracy como la métrica de evaluación, a modo de prevenir
la influencia de desbalances en la distribución de clases en los datos
* Utilizamos Cross Entropy Loss dado que se trata de un problema de clasificación multi-clase
* Agregamos un paso previo a la evaluación de las predicciones, que toma el valor máximo de la capa de salida (output layer) y asigna 1 al nodo que tiene el valor máximo y 0 en los otros casos. De este modo nos aseguramos de que haya una sola clase como output.


In [ ]:
embedding_size = 50
output_size = n_categories

In [ ]:
mlflow.set_experiment("an_amazing_CNN_experiment")

with mlflow.start_run():
    mlflow.log_param("model_name", "cnn")
    mlflow.log_param("freeze_embedding", True)
    mlflow.log_params({
        "filters_count": FILTERS_COUNT,
        "filters_length": FILTERS_LENGTH,
        "fc_size": 128
    })
    model = MeLiChallengeClassifier(path_glove,
                                    preprocess_train.dictionary,
                                    embedding_size,
                                    True,
                                    output_size)
    loss = nn.CrossEntropyLoss() # Usamos Cross Entropy Loss al tratarse de un multi-class classification problem
    optimizer = optim.Adam(model.parameters(), lr=1e-3, weight_decay=1e-5)
    for epoch in trange(3):
        model.train()
        running_loss = []
        for idx, batch in enumerate(tqdm(train_loader)):
            optimizer.zero_grad()
            output = model(batch["data"])
            loss_value = loss(output, batch["target"])
            loss_value.backward()
            optimizer.step()
            running_loss.append(loss_value.item())        
        mlflow.log_metric("train_loss", sum(running_loss) / len(running_loss), epoch)
        
        model.eval()
        running_loss = []
        targets = []
        predictions = []
        for batch in tqdm(test_loader):
            output = model(batch["data"])
            running_loss.append(
                loss(output, batch["target"]).item()
            )
            targets.extend(batch["target"].numpy())
            # Tomamos el índice del valor máximo para cada predicción
            x = np.argmax(output.squeeze().detach().numpy(), axis=1)
            predictions.extend(x)
        mlflow.log_metric("test_loss", sum(running_loss) / len(running_loss), epoch)
        mlflow.log_metric("test_balanced_accuracy", balanced_accuracy_score(targets, predictions), epoch)
        print(balanced_accuracy_score(targets, predictions))
    
    with tempfile.TemporaryDirectory() as tmpdirname:
        targets = []
        predictions = []
        for batch in tqdm(test_loader):
            output = model(batch["data"])
            targets.extend(batch["target"].numpy())
            predictions.extend(output.squeeze().detach().numpy())
        pd.DataFrame({"prediction": predictions, "target": targets}).to_csv(
            f"{tmpdirname}/predictions.csv.gz", index=False
        )
        mlflow.log_artifact(f"{tmpdirname}/predictions.csv.gz")

2022/11/22 00:55:14 INFO mlflow.tracking.fluent: Experiment with name 'an_amazing_CNN_experiment' does not exist. Creating a new experiment.


  0%|          | 0/3 [00:00<?, ?it/s]

  0%|          | 0/38245 [00:00<?, ?it/s]

  0%|          | 0/498 [00:00<?, ?it/s]

0.0019305014190504203


...

0.0016726589041428232


  0%|          | 0/38245 [00:00<?, ?it/s]

  0%|          | 0/498 [00:00<?, ?it/s]

0.001416792020735099


  0%|          | 0/498 [00:00<?, ?it/s]

In [ ]:
!zip -r /content/mlruns.zip /content/mlruns

  adding: content/mlruns/ (stored 0%)
  adding: content/mlruns/.trash/ (stored 0%)
  adding: content/mlruns/1/ (stored 0%)
  adding: content/mlruns/1/meta.yaml (deflated 26%)
  adding: content/mlruns/1/2f619b187fb44c70a90ef82ebc75ac0d/ (stored 0%)
  adding: content/mlruns/1/2f619b187fb44c70a90ef82ebc75ac0d/metrics/ (stored 0%)
  adding: content/mlruns/1/2f619b187fb44c70a90ef82ebc75ac0d/metrics/test_loss (deflated 35%)
  adding: content/mlruns/1/2f619b187fb44c70a90ef82ebc75ac0d/metrics/train_loss (deflated 34%)
  adding: content/mlruns/1/2f619b187fb44c70a90ef82ebc75ac0d/metrics/test_balanced_accuracy (deflated 36%)
  adding: content/mlruns/1/2f619b187fb44c70a90ef82ebc75ac0d/params/ (stored 0%)
  adding: content/mlruns/1/2f619b187fb44c70a90ef82ebc75ac0d/params/freeze_embedding (stored 0%)
  adding: content/mlruns/1/2f619b187fb44c70a90ef82ebc75ac0d/params/filters_count (stored 0%)
  adding: content/mlruns/1/2f619b187fb44c70a90ef82ebc75ac0d/params/filters_length (stored 0%)
  adding: conte

In [ ]:
from google.colab import files
files.download("/content/mlruns.zip")

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [1]:
ZIP_PATH = "images"

!zip -r {ZIP_PATH + ".zip"} {ZIP_PATH}

	zip warning: name not matched: images

zip error: Nothing to do! (try: zip -r images.zip . -i images)
